# Selector Group Chat

In [2]:
from typing import List, Sequence

from autogen_agentchat.agents import AssistantAgent, UserProxyAgent
from autogen_agentchat.conditions import MaxMessageTermination, TextMentionTermination
from autogen_agentchat.messages import BaseAgentEvent, BaseChatMessage
from autogen_agentchat.teams import SelectorGroupChat
from autogen_agentchat.ui import Console
from autogen_ext.models.openai import OpenAIChatCompletionClient

In [3]:
# Note: This example uses mock tools instead of real APIs for demonstration purposes
def search_web_tool(query: str) -> str:
    if "2006-2007" in query:
        return """Here are the total points scored by Miami Heat players in the 2006-2007 season:
        Udonis Haslem: 844 points
        Dwayne Wade: 1397 points
        James Posey: 550 points
        ...
        """
    elif "2007-2008" in query:
        return "The number of total rebounds for Dwayne Wade in the Miami Heat season 2007-2008 is 214."
    elif "2008-2009" in query:
        return "The number of total rebounds for Dwayne Wade in the Miami Heat season 2008-2009 is 398."
    return "No data found."


def percentage_change_tool(start: float, end: float) -> float:
    return ((end - start) / start) * 100

In [4]:
text_mention_termination = TextMentionTermination("TERMINATE")
max_messages_termination = MaxMessageTermination(max_messages=25)
termination = text_mention_termination | max_messages_termination

In [5]:
selector_prompt = """
Select an agent to perform task.

{roles}

Current conversation context:
{history}

Read the above conversation, then select an agent from {participants} to perform the next task.
When the task is complete, let the user approve or disapprove the task.
""".strip()

In [6]:
import os,sys
from dotenv import load_dotenv


def configure():
    load_dotenv()
    return os.getenv("OPENROUTER_API_KEY")

model_client = OpenAIChatCompletionClient(
    model="GLM-4.5-Flash",
    api_key=configure(),
    # base_url="...",
    base_url="https://api.z.ai/api/paas/v4/",
    temperature=0.8,
    # max_tokens=20000,
    model_info={
        "vision": False,
        "function_calling": True,
        "json_output": True,
        "family": "unknown",
        "structured_output": True,
    },
)

planning_agent = AssistantAgent(
    "PlanningAgent",
    description="An agent for planning tasks, this agent should be the first to engage when given a new task.",
    model_client=model_client,
    system_message="""
    You are a planning agent.
    Your job is to break down complex tasks into smaller, manageable subtasks.
    Your team members are:
        WebSearchAgent: Searches for information
        DataAnalystAgent: Performs calculations

    You only plan and delegate tasks - you do not execute them yourself.

    When assigning tasks, use this format:
    1. <agent> : <task>

    After all tasks are complete, summarize the findings and end with "TERMINATE".
    """,
)

web_search_agent = AssistantAgent(
    "WebSearchAgent",
    description="An agent for searching information on the web.",
    tools=[search_web_tool],
    model_client=model_client,
    system_message="""
    You are a web search agent.
    Your only tool is search_tool - use it to find information.
    You make only one search call at a time.
    Once you have the results, you never do calculations based on them.
    """,
)

data_analyst_agent = AssistantAgent(
    "DataAnalystAgent",
    description="An agent for performing calculations.",
    model_client=model_client,
    tools=[percentage_change_tool],
    system_message="""
    You are a data analyst.
    Given the tasks you have been assigned, you should analyze the data and provide results using the tools provided.
    If you have not seen the data, ask for it.
    """,
)

In [7]:
task = "Who was the Miami Heat player with the highest points in the 2006-2007 season, and what was the percentage change in his total rebounds between the 2007-2008 and 2008-2009 seasons?"

In [9]:
user_proxy_agent = UserProxyAgent("UserProxyAgent", description="A proxy for the user to approve or disapprove tasks.")


def selector_func_with_user_proxy(messages: Sequence[BaseAgentEvent | BaseChatMessage]) -> str | None:
    if messages[-1].source != planning_agent.name and messages[-1].source != user_proxy_agent.name:
        # Planning agent should be the first to engage when given a new task, or check progress.
        return planning_agent.name
    if messages[-1].source == planning_agent.name:
        if messages[-2].source == user_proxy_agent.name and "APPROVE" in messages[-1].content.upper():  # type: ignore
            # User has approved the plan, proceed to the next agent.
            return None
        # Use the user proxy agent to get the user's approval to proceed.
        return user_proxy_agent.name
    if messages[-1].source == user_proxy_agent.name:
        # If the user does not approve, return to the planning agent.
        if "APPROVE" not in messages[-1].content.upper():  # type: ignore
            return planning_agent.name
    return None

team = SelectorGroupChat(
    [planning_agent, web_search_agent, data_analyst_agent, user_proxy_agent],
    model_client=model_client,
    termination_condition=termination,
    selector_prompt=selector_prompt,
    selector_func=selector_func_with_user_proxy,
    allow_repeated_speaker=True,
)

# Reset the previous agents and run the chat again with the user proxy agent and selector function.
await team.reset()

await Console(team.run_stream(task=task))

---------- TextMessage (user) ----------
Who was the Miami Heat player with the highest points in the 2006-2007 season, and what was the percentage change in his total rebounds between the 2007-2008 and 2008-2009 seasons?


---------- ThoughtEvent (PlanningAgent) ----------
I need to break down this complex task into smaller, manageable subtasks. The question has two parts:

1. Find who was the Miami Heat player with the highest points in the 2006-2007 season
2. Find the percentage change in that player's total rebounds between the 2007-2008 and 2008-2009 seasons

To accomplish this, I'll need to:

1. First search for information about Miami Heat players' points in the 2006-2007 season
2. Identify which player had the highest points
3. Search for that player's rebound statistics for the 2007-2008 season
4. Search for that player's rebound statistics for the 2008-2009 season
5. Calculate the percentage change in rebounds between those two seasons

Let me assign these tasks to the appropriate agents:

For the search tasks, I'll use the WebSearchAgent:
- Search for Miami Heat players' points in the 2006-2007 season
- Search for the identified player's rebounds in 2007-2008
- Search for the identified player'

TaskResult(messages=[TextMessage(id='089ee20e-dcb2-4f9f-87c7-6d5dad381605', source='user', models_usage=None, metadata={}, created_at=datetime.datetime(2025, 8, 7, 7, 29, 55, 790977, tzinfo=datetime.timezone.utc), content='Who was the Miami Heat player with the highest points in the 2006-2007 season, and what was the percentage change in his total rebounds between the 2007-2008 and 2008-2009 seasons?', type='TextMessage'), ThoughtEvent(id='5f1fc02c-7a57-42a8-8096-20beefd97c0f', source='PlanningAgent', models_usage=None, metadata={}, created_at=datetime.datetime(2025, 8, 7, 7, 30, 1, 840633, tzinfo=datetime.timezone.utc), content="I need to break down this complex task into smaller, manageable subtasks. The question has two parts:\n\n1. Find who was the Miami Heat player with the highest points in the 2006-2007 season\n2. Find the percentage change in that player's total rebounds between the 2007-2008 and 2008-2009 seasons\n\nTo accomplish this, I'll need to:\n\n1. First search for info

Now we run the team with a task to find information about an NBA player.

As we can see, after the Web Search Agent conducts the necessary searches and the Data Analyst Agent completes the necessary calculations, we find that Dwayne Wade was the Miami Heat player with the highest points in the 2006-2007 season, and the percentage change in his total rebounds between the 2007-2008 and 2008-2009 seasons is 85.98%!

## Custom Selector Function

Often times we want better control over the selection process.
To this end, we can set the `selector_func` argument with a custom selector function to override the default model-based selection.
This allows us to implement more complex selection logic and state-based transitions.

For instance, we want the Planning Agent to speak immediately after any specialized agent to check the progress.

```{note}
Returning `None` from the custom selector function will use the default model-based selection.
``` 

```{note}
Custom selector functions are not [serialized](https://microsoft.github.io/autogen/stable/user-guide/agentchat-user-guide/serialize-components.html) when `.dump_component()` is called on the SelectorGroupChat team . If you need to serialize team configurations with custom selector functions, consider implementing custom workflows and serialization logic.
```

In [10]:
def selector_func(messages: Sequence[BaseAgentEvent | BaseChatMessage]) -> str | None:
    if messages[-1].source != planning_agent.name:
        return planning_agent.name
    return None


# Reset the previous team and run the chat again with the selector function.
await team.reset()
team = SelectorGroupChat(
    [planning_agent, web_search_agent, data_analyst_agent],
    model_client=model_client,
    termination_condition=termination,
    selector_prompt=selector_prompt,
    allow_repeated_speaker=True,
    selector_func=selector_func,
)

await Console(team.run_stream(task=task))

---------- TextMessage (user) ----------
Who was the Miami Heat player with the highest points in the 2006-2007 season, and what was the percentage change in his total rebounds between the 2007-2008 and 2008-2009 seasons?
---------- ThoughtEvent (PlanningAgent) ----------

This question requires me to find specific basketball statistics about the Miami Heat player. Let me break this down into smaller subtasks:

1. First, I need to identify who was the Miami Heat player with the highest points in the 2006-2007 season.
2. Then, I need to find that player's total rebounds in the 2007-2008 season.
3. Next, I need to find that player's total rebounds in the 2008-2009 season.
4. Finally, I need to calculate the percentage change in rebounds between those two seasons.

Let me assign these tasks to the appropriate agents:

For the first three tasks, I need to find specific basketball statistics, which would be best handled by the WebSearchAgent who can search for this information.

The final t

TaskResult(messages=[TextMessage(id='b3157bb5-fb54-4d61-85cc-fa325cbe6406', source='user', models_usage=None, metadata={}, created_at=datetime.datetime(2025, 8, 7, 7, 32, 40, 668789, tzinfo=datetime.timezone.utc), content='Who was the Miami Heat player with the highest points in the 2006-2007 season, and what was the percentage change in his total rebounds between the 2007-2008 and 2008-2009 seasons?', type='TextMessage'), ThoughtEvent(id='5719a231-2d0e-401d-8b6d-91d0768dd9d7', source='PlanningAgent', models_usage=None, metadata={}, created_at=datetime.datetime(2025, 8, 7, 7, 32, 48, 659359, tzinfo=datetime.timezone.utc), content="\nThis question requires me to find specific basketball statistics about the Miami Heat player. Let me break this down into smaller subtasks:\n\n1. First, I need to identify who was the Miami Heat player with the highest points in the 2006-2007 season.\n2. Then, I need to find that player's total rebounds in the 2007-2008 season.\n3. Next, I need to find that

You can see from the conversation log that the Planning Agent always speaks immediately after the specialized agents.

```{tip}
Each participant agent only makes one step (executing tools, generating a response, etc.)
on each turn. 
If you want an {py:class}`~autogen_agentchat.agents.AssistantAgent` to repeat
until it stop returning a {py:class}`~autogen_agentchat.messages.ToolCallSummaryMessage`
when it has finished running all the tools it needs to run, you can do so by
checking the last message and returning the agent if it is a
{py:class}`~autogen_agentchat.messages.ToolCallSummaryMessage`.
```

## Custom Candidate Function

One more possible requirement might be to automatically select the next speaker from a filtered list of agents.
For this, we can set `candidate_func` parameter with a custom candidate function to filter down the list of potential agents for speaker selection for each turn of groupchat.

This allow us to restrict speaker selection to a specific set of agents after a given agent.


```{note}
The `candidate_func` is only valid if `selector_func` is not set.
Returning `None` or an empty list `[]` from the custom candidate function will raise a `ValueError`.
```

In [11]:
def candidate_func(messages: Sequence[BaseAgentEvent | BaseChatMessage]) -> List[str]:
    # keep planning_agent first one to plan out the tasks
    if messages[-1].source == "user":
        return [planning_agent.name]

    # if previous agent is planning_agent and if it explicitely asks for web_search_agent
    # or data_analyst_agent or both (in-case of re-planning or re-assignment of tasks)
    # then return those specific agents
    last_message = messages[-1]
    if last_message.source == planning_agent.name:
        participants = []
        if web_search_agent.name in last_message.to_text():
            participants.append(web_search_agent.name)
        if data_analyst_agent.name in last_message.to_text():
            participants.append(data_analyst_agent.name)
        if participants:
            return participants  # SelectorGroupChat will select from the remaining two agents.

    # we can assume that the task is finished once the web_search_agent
    # and data_analyst_agent have took their turns, thus we send
    # in planning_agent to terminate the chat
    previous_set_of_agents = set(message.source for message in messages)
    if web_search_agent.name in previous_set_of_agents and data_analyst_agent.name in previous_set_of_agents:
        return [planning_agent.name]

    # if no-conditions are met then return all the agents
    return [planning_agent.name, web_search_agent.name, data_analyst_agent.name]


# Reset the previous team and run the chat again with the selector function.
await team.reset()
team = SelectorGroupChat(
    [planning_agent, web_search_agent, data_analyst_agent],
    model_client=model_client,
    termination_condition=termination,
    candidate_func=candidate_func,
)

await Console(team.run_stream(task=task))

---------- TextMessage (user) ----------
Who was the Miami Heat player with the highest points in the 2006-2007 season, and what was the percentage change in his total rebounds between the 2007-2008 and 2008-2009 seasons?
---------- ThoughtEvent (PlanningAgent) ----------

To answer this question, I need to find:

1. Who was the Miami Heat player with the highest points in the 2006-2007 season
2. What was that player's total rebounds in the 2007-2008 season
3. What was that player's total rebounds in the 2008-2009 season
4. Calculate the percentage change in rebounds between those two seasons

Let me break this down into tasks:

1. WebSearchAgent: Find the Miami Heat player with the highest points in the 2006-2007 season
2. WebSearchAgent: Find that player's total rebounds in the 2007-2008 season
3. WebSearchAgent: Find that player's total rebounds in the 2008-2009 season
4. DataAnalystAgent: Calculate the percentage change in rebounds between the 2007-2008 and 2008-2009 seasons

Once 

TaskResult(messages=[TextMessage(id='ca5d037e-a34b-4738-acc7-8ff46c55d963', source='user', models_usage=None, metadata={}, created_at=datetime.datetime(2025, 8, 7, 7, 35, 0, 209938, tzinfo=datetime.timezone.utc), content='Who was the Miami Heat player with the highest points in the 2006-2007 season, and what was the percentage change in his total rebounds between the 2007-2008 and 2008-2009 seasons?', type='TextMessage'), ThoughtEvent(id='5c5c6aad-2351-4275-bd30-50a6866feb38', source='PlanningAgent', models_usage=None, metadata={}, created_at=datetime.datetime(2025, 8, 7, 7, 35, 5, 230251, tzinfo=datetime.timezone.utc), content="\nTo answer this question, I need to find:\n\n1. Who was the Miami Heat player with the highest points in the 2006-2007 season\n2. What was that player's total rebounds in the 2007-2008 season\n3. What was that player's total rebounds in the 2008-2009 season\n4. Calculate the percentage change in rebounds between those two seasons\n\nLet me break this down into

You can see from the conversation log that the Planning Agent returns to conversation once the Web Search Agent and Data Analyst Agent took their turns and it finds that the task was not finished as expected so it called the WebSearchAgent again to get rebound values and then called DataAnalysetAgent to get the percentage change.

## User Feedback

We can add {py:class}`~autogen_agentchat.agents.UserProxyAgent` to the team to
provide user feedback during a run.
See [Human-in-the-Loop](./tutorial/human-in-the-loop.ipynb) for more details
about {py:class}`~autogen_agentchat.agents.UserProxyAgent`.

To use the {py:class}`~autogen_agentchat.agents.UserProxyAgent` in the 
web search example, we simply add it to the team and update the selector function
to always check for user feedback after the planning agent speaks.
If the user responds with `"APPROVE"`, the conversation continues, otherwise,
the planning agent tries again, until the user approves.

In [ ]:
user_proxy_agent = UserProxyAgent("UserProxyAgent", description="A proxy for the user to approve or disapprove tasks.")


def selector_func_with_user_proxy(messages: Sequence[BaseAgentEvent | BaseChatMessage]) -> str | None:
    if messages[-1].source != planning_agent.name and messages[-1].source != user_proxy_agent.name:
        # Planning agent should be the first to engage when given a new task, or check progress.
        return planning_agent.name
    if messages[-1].source == planning_agent.name:
        if messages[-2].source == user_proxy_agent.name and "APPROVE" in messages[-1].content.upper():  # type: ignore
            # User has approved the plan, proceed to the next agent.
            return None
        # Use the user proxy agent to get the user's approval to proceed.
        return user_proxy_agent.name
    if messages[-1].source == user_proxy_agent.name:
        # If the user does not approve, return to the planning agent.
        if "APPROVE" not in messages[-1].content.upper():  # type: ignore
            return planning_agent.name
    return None


# Reset the previous agents and run the chat again with the user proxy agent and selector function.
await team.reset()
team = SelectorGroupChat(
    [planning_agent, web_search_agent, data_analyst_agent, user_proxy_agent],
    model_client=model_client,
    termination_condition=termination,
    selector_prompt=selector_prompt,
    selector_func=selector_func_with_user_proxy,
    allow_repeated_speaker=True,
)

await Console(team.run_stream(task=task))

---------- user ----------
Who was the Miami Heat player with the highest points in the 2006-2007 season, and what was the percentage change in his total rebounds between the 2007-2008 and 2008-2009 seasons?


---------- PlanningAgent ----------
To address the user's query, we will need to perform the following tasks:

1. Identify the Miami Heat player with the highest points in the 2006-2007 season.
2. Find the total rebounds for that player in the 2007-2008 season.
3. Find the total rebounds for that player in the 2008-2009 season.
4. Calculate the percentage change in the total rebounds between the 2007-2008 and 2008-2009 seasons.

Let's assign these tasks:

1. **WebSearchAgent**: Identify the Miami Heat player with the highest points in the 2006-2007 season.
   
(Task 2 and 3 depend on the result of Task 1. We'll proceed with Tasks 2 and 3 once Task 1 is complete.)
---------- UserProxyAgent ----------
approve
---------- WebSearchAgent ----------
[FunctionCall(id='call_0prr3fUnG5CtisUG7QeygW0w', arguments='{"query":"Miami Heat highest points scorer 2006-2007 NBA season"}', name='search_web_tool')]
---------- WebSearchAgent ----------
[FunctionExecutionResult(content='Here are the total po

TaskResult(messages=[TextMessage(source='user', models_usage=None, content='Who was the Miami Heat player with the highest points in the 2006-2007 season, and what was the percentage change in his total rebounds between the 2007-2008 and 2008-2009 seasons?', type='TextMessage'), TextMessage(source='PlanningAgent', models_usage=RequestUsage(prompt_tokens=161, completion_tokens=166), content="To address the user's query, we will need to perform the following tasks:\n\n1. Identify the Miami Heat player with the highest points in the 2006-2007 season.\n2. Find the total rebounds for that player in the 2007-2008 season.\n3. Find the total rebounds for that player in the 2008-2009 season.\n4. Calculate the percentage change in the total rebounds between the 2007-2008 and 2008-2009 seasons.\n\nLet's assign these tasks:\n\n1. **WebSearchAgent**: Identify the Miami Heat player with the highest points in the 2006-2007 season.\n   \n(Task 2 and 3 depend on the result of Task 1. We'll proceed with

Now, the user's feedback is incorporated into the conversation flow,
and the user can approve or reject the planning agent's decisions.

```{tip}
For more guidance on how to prompt reasoning models, see the
Azure AI Services Blog on [Prompt Engineering for OpenAI's O1 and O3-mini Reasoning Models](https://techcommunity.microsoft.com/blog/azure-ai-services-blog/prompt-engineering-for-openai%E2%80%99s-o1-and-o3-mini-reasoning-models/4374010)
```